<a href="https://colab.research.google.com/github/jepennerhahn/X-ray_Absorbance/blob/main/X_ray_absorbanceV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#This notebook will use xraylib to calculate sample properties

In [ ]:
#@title First we need to install a few things. 

#@markdown Please execute this cell by pressing the _Play_ button 
#@markdown on the left.  This will take a few minutes.
%%capture

'''Copyright (c) 2022 J.E. Penner-Hahn

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:
The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.
THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.'''

import os
#Load a bash file that will install miniconda and xraylib

if not os.path.isdir('X-ray_Absorbance/'):
    !git clone "https://github.com/jepennerhahn/X-ray_Absorbance.git"

import subprocess
permissions = subprocess.run(['chmod','a+x','X-ray_Absorbance/install_xraylib.sh'])
%env PYTHONPATH=
!bash X-ray_Absorbance/install_xraylib.sh
import sys
_ = sys.path.append("/usr/local/lib/python3.7/site-packages")
import xraylib as xr
import math
import numpy as np
import matplotlib.pyplot as plt

In [37]:
#@title Now define a few functions. 
xr.XRayInit()
edgeList = ['K', 'L1','L2','L3','M1','M2','M3','M4','M5']
def fracDen(MW,Conc,wtFrac):
    #For a molecule with molecular weight MW and molar concentration Conc in water, 
    #This will convert the weight fraction of an element into fractional density (g/cm3) of that element
    #Assumes a density of 1 g/cm3
    F = Conc*MW*wtFrac/1000.
    return F

#Define our sliders
import ipywidgets as widgets
s1 = widgets.widgets.Text(
    value='H2O',
    placeholder='Enter chemical formula, e.g. H2 O',
    description='Sample:',
    disabled=False)
conc = widgets.FloatText(description = 'Conc. (mM)', value = 1)
dens = widgets.FloatText(description = 'Density (g/cm2)', value = 2, style = {'description_width': 'initial'})
thickness = widgets.FloatText(description = 'Thickness (um)', value = 1, style = {'description_width': 'initial'})
soln = widgets.Checkbox(description = 'Solution', value = False)
Range = widgets.FloatRangeSlider(description = 'Plot range (keV)', 
    min = 0.1,
    max = 25.,
    step = 0.1,
    value = (5,15),
    style = {'description_width': 'initial'},
    layout=widgets.Layout(width='600px'))
def update_density(*args):
    if soln.value == True:
        dens.value = 1.0
    else:
        dens.value = 2.0
soln.observe(update_density,'value')

ui = widgets.VBox([widgets.HBox([s1, dens,thickness]),widgets.HBox([soln, conc]),
                   Range])

def plotAbs(comp,E,solution,thickness,dens):
  [isSoln,conc] = solution
#conc is concentration in mM.  This is unused if isSoln == False
  Energy = np.arange(E[0],E[1],(E[1]-E[0])/1000.) #Energy range for plot of absorption coefficient
#The following check is not required if we use a slider rather than text boxes for range
  if E[1] > 100: #If the highest energy is > 100, assume that these are eV, not keV
    xlab = 'Energy (eV)'
    Energy = Energy/1000.
    xplot = Energy*1000.
  else:
    xlab = 'Energy (keV)'
    xplot = Energy

  Abs = np.array([]) #Accumulator for cross-section (cm2/g) of the solid
  SolAbs = np.array([]) #Accumulator for cross-section (cm2/g) of the solution
  for En in Energy:
    A = 0.
    SA = 16./18.*xr.CS_Total(8,En)+2./18.*xr.CS_Total(1,En) #Absorbance of water
    for i,El in enumerate(comp['Elements']):
        A += xr.CS_Total(El, En)*comp['massFractions'][i]
        SA += xr.CS_Total(El, En)*fracDen(comp['molarMass'],conc/1000.,comp['massFractions'][i])
    Abs = np.append(Abs,A) #Don't really need Abs and SolAbs, but maybe we'll add 2 more plots, comparing solid and soon
    SolAbs = np.append(SolAbs,SA)
  Abs = Abs * thickness * dens * 1e-4 # The 1e-4 is to convert thickness to cm
  SolAbs = SolAbs * thickness * dens * 1e-4 
  if isSoln:
    yplot = SolAbs
  else:
    yplot = Abs
  ylab = 'Absorbance'
  fig,axs = plt.subplots(nrows = 1,ncols = 2, figsize = (12,6))
  axs[0].plot(xplot,yplot)
  axs[0].set_yscale('log')
  axs[0].set_ylabel(ylab)
  axs[0].set_xlabel(xlab)
  axs[0].set_title('Log scale absorbance')
  # axs[0].locator_params(axis="x", nbins=5)
  axs[0].grid(which = 'both')
  axs[1].plot(xplot,yplot)
  axs[1].set_ylabel(ylab)
  axs[1].set_xlabel(xlab)
  axs[1].set_title('Linear scale absorbance')
  # axs[1].locator_params(axis="x", nbins=5)
  axs[1].grid()
  plt.show()
  return


def plotXAS(s1,soln,Range,conc,thickness,dens):
    try:
      p_sample = xr.CompoundParser(s1.replace(" ",""))
      print("Sample contains {} atoms and {} elements with a molar mass of {:6.2f} g/mol"\
            .format(int(p_sample['nAtomsAll']), p_sample['nElements'], p_sample['molarMass']))
      for i in range(p_sample['nElements']):
          print("Element {} (Z={}): {:4.2f}% by mass with {} atoms".format\
                (xr.AtomicNumberToSymbol(p_sample['Elements'][i]),p_sample['Elements'][i], \
                p_sample['massFractions'][i]*100.0, int(p_sample['nAtoms'][i])))
      #Assume that the heaviest element (last in list) is the one that is of interest if no value entered
    except:
      print('Enter valid sample')
      return

    Zabs = p_sample['Elements'][-1]
    edgeValues = {}
    for i in range(9):  #Can be as high as 8, to get the M edges also
        try:
            Evalue = xr.EdgeEnergy(Zabs, i)*1000
            edgeValues[i] = Evalue
        except:
            continue
    print('{} has edges (in eV) at:'.format(xr.AtomicNumberToSymbol(Zabs)))
    print('  {} = {:.1f}'.format(edgeList[0],edgeValues[0]))
    if len(edgeValues) > 1: #Have at least one L edge
        maxindex = min(4,len(edgeValues))
        print(*['  ' + edgeList[n] + ' = ' + '{:.1f}'
        .format(edgeValues[n]) for n in range(maxindex-1,0,-1)])
    if len(edgeValues) > 4: #Have at least one M edge
        maxindex = min(9,len(edgeValues))
        print(*['  ' + edgeList[n] + ' = ' + '{:.1f}'
        .format(edgeValues[n]) for n in range(maxindex-1,3,-1)])

    plotAbs(p_sample,Range,[soln,conc],thickness,dens)
    return

In [40]:
#@title Enter sample details 

#@markdown Enter composition (for example, as C2 H6 O for ethanol)<br> 
#@markdown Default density is 2 for solids, 1 for solutions<br>
#@markdown Concentration is only used if solution box is checked<br>
#@markdown Energies are assumed to be keV unless E_high > 50, then eV
out = widgets.interactive_output(plotXAS, 
        {'s1': s1, 'soln': soln, 'Range': Range, 'conc': conc, 
         'thickness': thickness, 'dens': dens})
display(ui,out)

Output()